# Simulador Recursivo
CC BY : https://creativecommons.org/licenses/by/4.0/deed.pt_BR

Este simulador é capaz de gerar imagens sísmicas com e sem múltiplas.

Thiago Pacheco Carneiro

In [1]:
import numpy as np
from scipy import signal
import pandas as pd
import matplotlib.pyplot as plt
#import pathos
from typing import Union

from tqdm.notebook import tqdm, trange
import time

In [2]:
class Camada():
    def __init__(self, profundidade: float, velocidade: float, densidade: float) -> None:
        self.prof = profundidade
        self.vel = velocidade
        self.rho = densidade
        self.imped = velocidade*densidade # impedância
        self.tempo = profundidade/velocidade

In [3]:
class Modelo():
    def __init__(self, camadas: list[Camada]) -> None:
        self.camadas = camadas
        r_ida = []
        t_ida = []
        r_vinda = []
        t_vinda = []
        for i in range(len(camadas)-1):
            z1 = camadas[i].imped
            z2 = camadas[i+1].imped
            r_i = (z2 - z1) / (z2 + z1)
            t_i = 2 * z1 / (z2 + z1)
            r_ida.append(r_i)
            t_ida.append(t_i)
            r_v = (z1 - z2) / (z2 + z1)
            t_v = 2 * z2 / (z2 + z1)
            r_vinda.append(r_v)
            t_vinda.append(t_v)
        self.nz = np.sum([camada.prof for camada in camadas])
        self.reflex_ida = r_ida
        self.transm_ida = t_ida
        self.reflex_vinda = r_vinda
        self.transm_vinda = t_vinda
        t = 0
        tempos = []
        for i in range(len(camadas)):
            t += camadas[i].tempo
            tempos.append(t)
        self.tempo_retorno = tempos
    def pulsar(self, i: int, t: float, pulso: float, sentido: int, t_max: int, limpo: bool = False) -> Union[list[dict[str,float]],None]:
        ida = vinda = None
        pulso *= (1-5e-4)**self.camadas[i].prof     #dissipação com a distância
        t_i = t + self.camadas[i].tempo
        retorno = []
        if sentido == 1:
            t_retorno = t_i + self.tempo_retorno[i]
        else:
            t_retorno = t + self.tempo_retorno[i]
        if t_retorno >= t_max or abs(pulso)<1e-6:
            return None
        if sentido == 1 and i<len(self.reflex_ida):
            ida = self.pulsar(i + 1, t_i, pulso * self.transm_ida[i], 1, t_max, limpo)
            vinda = self.pulsar(i, t_i, pulso * self.reflex_ida[i], -1, t_max, limpo)
        if sentido == -1:
            if i == 0:
                retorno.append({'instante':t_i, 'pulso': pulso})
                if limpo == False:
                    ida = self.pulsar(i, t_i, -pulso, 1, t_max, limpo)
            else:
                vinda = self.pulsar(i - 1, t_i, pulso * self.transm_vinda[i - 1], -1, t_max, limpo)
                if limpo == False:
                    ida = self.pulsar(i, t_i, pulso * self.reflex_vinda[i - 1], 1, t_max, limpo)
        if ida is not None:
            retorno += ida
        if vinda is not None:
            retorno += vinda
        return retorno
    def obter_pulsos(self, t_max: int = 3, limpo: bool = False) -> pd.DataFrame:
        p = self.pulsar(i = 0, t = 0., pulso=1., sentido=1, t_max = t_max, limpo = limpo)
        if p is None:
            return pd.DataFrame([{'instante': 0, 'pulso' : 0},{'instante': 1, 'pulso' : 0}]).groupby('instante').sum()
        return pd.DataFrame(p).groupby('instante').sum()
    def adquirir_sismica(self, t_max: int = 3, n_linhas: int = 1e2, limpo: bool = False) -> np.ndarray:
        sis = np.zeros((int(n_linhas),))
        pulsos = self.obter_pulsos(t_max, limpo)
        pulsos['z']=(pulsos.index*n_linhas/t_max).astype(int)
        pulsos_z = pulsos.groupby('z').sum()
        for z, linha in pulsos_z.iterrows():
            if z < len(sis):
                sis[z] += linha['pulso']
        return sis
    def convoluida(self, t_max: int = 3, n_linhas: int = 1e2, pulso = -signal.ricker(10, 1), limpo: bool = False) -> np.ndarray:
        sinal = self.adquirir_sismica(t_max, n_linhas, limpo)
        convolucao = signal.convolve(sinal, pulso, mode = 'same')
        return convolucao*(convolucao>1e-6)
    def rasterizar(self) -> dict[str,np.ndarray]:
        v = np.zeros((self.nz,))
        rho = np.zeros((self.nz,))
        zi = 0
        for camada in self.camadas:
            zf = zi + camada.prof
            v[zi:zf] = camada.vel
            rho[zi:zf] = camada.rho
            zi = zf
        return {'velocidades': v, 'densidades': rho}
    def plot(self, t_max: int = 3, n_linhas: int = 256, titulo: str = '') -> None:
        rast = self.rasterizar()
        v = rast['velocidades']
        rho = rast['densidades']
        sis = self.adquirir_sismica(t_max=t_max,n_linhas=n_linhas)
        sis_limpo = self.adquirir_sismica(t_max=t_max,n_linhas=n_linhas, limpo=True)
        fig, ax = plt.subplots(4,1, figsize=(10,14))
        fig.suptitle(titulo)
        ax[0].set_title('Velocidade')
        ax[0].set_xlabel('Profundidade (m)')
        ax[0].set_ylabel('Velocidade (m/s)')
        ax[1].set_title('Densidade')
        ax[1].set_xlabel('Profundidade (m)')
        ax[1].set_ylabel('Densidade (g/cm3)')
        ax[2].set_title('Sismograma')
        ax[2].set_xlabel('Tempo (s)')
        ax[2].set_ylabel('Pulso')
#        ax[2].set_yscale('log')
        ax[3].set_title('Sismograma "limpo"')
        ax[3].set_xlabel('Tempo (s)')
        ax[3].set_ylabel('Pulso')
#        ax[3].set_yscale('log')
        ax[0].plot(v)
        ax[1].plot(rho)
        # ax[2].plot(sis)
        # ax[3].plot(sis_limpo)
        # ax[2].scatter([x for x in range(len(sis))],sis)
        # ax[3].scatter([x for x in range(len(sis_limpo))],sis_limpo)
        # ax[2].bar([x for x in range(len(sis))],sis)
        # ax[3].bar([x for x in range(len(sis_limpo))],sis_limpo)
        ax[2].step([x for x in range(len(sis))],sis)
        ax[3].step([x for x in range(len(sis_limpo))],sis_limpo)
        plt.tight_layout()
    def print(self, t_max: int = 3, n_linhas: int = 256):
        sis = self.obter_pulsos(t_max=t_max)
        sis_limpo = self.obter_pulsos(t_max=t_max, limpo = True)
        for i in range(len(sis)):
            if sis.index[i] in sis_limpo.index:
                print(sis.index[i],'\t',sis.iloc[i].pulso,'\t',sis_limpo.loc[sis.index[i]].pulso)
            else:
                print(sis.index[i],'\t',sis.iloc[i].pulso)

In [4]:
class Modelo2D():
    def __init__(self, camadas2D: list[list[Camada]]) -> None:
        cols = []
        nz = 0
        for col in camadas2D:
            m_i = Modelo(col)
            cols.append(m_i)
            nz = m_i.nz if m_i.nz > nz else nz
        self.colunas=cols
        self.nz = nz
        self.nx = len(camadas2D)
        if self.nx != len(self.colunas):
            print(self.nx, '!=', len(self.colunas))
    def adquirir_sismica_2D(self, t_max: int = 3, n_linhas: int = 256, limpo: bool = False) -> np.ndarray:
        sis = []
        for coluna in self.colunas:
            sis.append(coluna.adquirir_sismica(t_max, n_linhas, limpo))
        sis = np.array(sis)
        return sis.T
    def convoluida_2D(self, t_max: int = 3, n_linhas: int = 1e2, limpo: bool = False, pulso = -signal.ricker(100, 1)) -> np.ndarray:
        sis = []
        for coluna in self.colunas:
            sis.append(coluna.convoluida(t_max, n_linhas, pulso, limpo))
        sis = np.array(sis)
        return sis.T
    def rasterizar_2D(self) -> dict[str,np.ndarray]:
        v2D = np.zeros((self.nx,self.nz))
        rho2D = np.zeros((self.nx,self.nz))
        for x in range(self.nx):
            col = self.colunas[x]
            r = col.rasterizar()
            v2D[x,:col.nz] = r['velocidades']
            rho2D[x,:col.nz] = r['densidades']
        return {'velocidades': v2D.T,
                'densidades': rho2D.T}
    def plot(self, t_max: int = 3, n_linhas: int = 256, titulo: str = '') -> None:
        rast = self.rasterizar_2D()
        v2D = rast['velocidades']
        rho2D = rast['densidades']
        sis = self.adquirir_sismica_2D(t_max=t_max,n_linhas=n_linhas)
        sis_limpo = self.adquirir_sismica_2D(t_max=t_max,n_linhas=n_linhas, limpo=True)
        fig, ax = plt.subplots(1,4, figsize=(14,10))
        fig.suptitle(titulo)
        ax[0].set_title('Velocidade')
        ax[0].set_xlabel('Distância (m)')
        ax[0].set_ylabel('Profundidade (m)')
        ax[1].set_title('Densidade')
        ax[1].set_xlabel('Distância (m)')
        ax[1].set_ylabel('Profundidade (m)')
        ax[2].set_title('Sismograma')
        ax[2].set_xlabel('Distância (m)')
        ax[2].set_ylabel('Tempo (s)')
        ax[3].set_title('Sismograma "limpo"')
        ax[3].set_xlabel('Distância (m)')
        ax[3].set_ylabel('Tempo (s)')
        ax[0].imshow(v2D, cmap='binary', aspect='auto')
        ax[1].imshow(rho2D, cmap='binary', aspect='auto')
        ax[2].imshow(sis, cmap='binary', aspect='auto',vmin=-1e-2, vmax=1e-2, extent=[0,self.nx,t_max,0])
        ax[3].imshow(sis_limpo, cmap='binary', aspect='auto',vmin=-1e-2, vmax=1e-2, extent=[0,self.nx,t_max,0])
        plt.tight_layout()

In [5]:
class ModeloParalelas(Modelo2D):
    def __init__(self, nx = 300, prof = (1000, 1000, 1000, 1000), v = (1500,2800,4500,3200), rho = (1, 2, 2.16, 2.6)) -> None:
        c2D = []
        c = []
        for i in range(len(prof)):
            camada = Camada(prof[i],v[i],rho[i])
            c.append(camada)
        for x in range(nx):
            c2D.append(c)
        super().__init__(c2D)
    def rasterizar_2D(self) -> dict[str,np.ndarray]:
        v2D = np.zeros((self.nx,self.nz))
        rho2D = np.zeros((self.nx,self.nz))
        col = self.colunas[0]
        r = col.rasterizar()
        for x in range(self.nx):
            v2D[x,:col.nz] = r['velocidades']
            rho2D[x,:col.nz] = r['densidades']
        return {'velocidades': v2D.T,
                'densidades': rho2D.T}
    def adquirir_sismica_2D(self, t_max: int = 3, n_linhas: int = 256, limpo: bool = True) -> np.ndarray:
        sis_x = self.colunas[0].adquirir_sismica(t_max, n_linhas, limpo)
        sis = [sis_x for _ in range(self.nx)]
        sis = np.array(sis)
        return sis.T

In [6]:
class ModeloDomo(Modelo2D):
    def __init__(self, nx = 300, v = (1500,2800,4500,3200), rho = (1, 2, 2.16, 2.6)) -> None:
        raio = nx/3
        c2D = []
        for x in range(nx):
            c = []
            agua = Camada(1000,v[0],rho[0])
            c.append(agua)
            cateto_x = abs(x - nx/2)
            cateto_z = int(7*np.sqrt(raio**2 - cateto_x**2)) if cateto_x < raio else 0
            posSal = Camada(1000-cateto_z,v[1],rho[1])
            c.append(posSal)
            if cateto_z > 0:
                sal = Camada(cateto_z,v[2],rho[2])
                c.append(sal)
            preSal = Camada(1000,v[3],rho[3])
            c.append(preSal)
            c2D.append(c)
        super().__init__(c2D)

In [7]:
class ModeloPreSal(Modelo2D):
    def __init__(self, nx = 300, v = (1500,2800,4500,5000,4500,2000,3200), rho = (1, 2, 2.16, 2.5, 2.16, 1.5, 2.6)) -> None:
        raio = nx/3
        c2D = []
        for x in range(nx):
            c = []
            agua = Camada(1000,v[0],rho[0])
            c.append(agua)
            cateto_x = abs(x - nx/2)
            cateto_z = int(np.sqrt(raio**2 - cateto_x**2)) if cateto_x < raio else 0
            posSal = Camada(1000-7*cateto_z,v[1],rho[1])
            c.append(posSal)
            if cateto_z > 0:
                sal1 = Camada(2*cateto_z,v[2],rho[2])
                c.append(sal1)
                sal2 = Camada(2*cateto_z,v[3],rho[3])
                c.append(sal2)
                sal3 = Camada(2*cateto_z,v[4],rho[4])
                c.append(sal3)
                oleo = Camada(cateto_z,v[5],rho[5])
                c.append(oleo)
            preSal = Camada(1000,v[6],rho[6])
            c.append(preSal)
            c2D.append(c)
        super().__init__(c2D)

In [8]:
class ModeloAleatorio(Modelo2D):
    def __init__(self, nx: int = 300, prof_min: int = 500, prof_max: int = 1000, v_min: int = 1500, v_max: int = 4500, rho_min: float = 1., rho_max: float = 2.5) -> None:
        c2D = []
        for x in range(nx):
            camadas = []
            for i in range(100):
                prof = np.random.randint(prof_min, prof_max)
                v =np.random.randint(v_min, v_max)
                rho = np.random.random()*(rho_max-rho_min)+rho_min
                camada = Camada(prof,v,rho)
                camadas.append(camada)
            c2D.append(camadas)
        super().__init__(c2D)

In [9]:
class ModeloSenoides(Modelo2D):
    def __init__(self, nx: int = 300, nz: int = 10000, prof_min: int = 500, prof_max: int = 1000, v_min: int = 1500, v_max: int = 4500, rho_min: float = 1., rho_max: float = 2.5) -> None:
        rast = np.zeros((nx,nz,2))
        rast[:,:,0] = v_min
        rast[:,:,1] = rho_min
        z = 0
        while z < nz-prof_max:
            d = np.random.randint(prof_min,prof_max)
            a = np.random.randint(0,prof_max)
            k = np.pi/np.random.randint(50,nx)
            fase = np.random.random()*np.pi*2
            velocidade = np.random.randint(v_min,v_max)
            rho = np.random.random()*(rho_max-rho_min)+rho_min
            z = z + d
            for x in range(nx):
                profundidade = int(nz-z-a*np.sin(k*x+fase))
                if profundidade < 0:
                    profundidade = 0
                rast[x,:profundidade,0] = velocidade
                rast[x,:profundidade,1] = rho
        c2D = []
        for x in range(nx):
            c = []
            z = 0
            while z < nz:
                velocidade = rast[x,z,0]
                rho = rast[x,z,1]
                profundidade = 0
                while (velocidade == rast[x,z+profundidade,0]) and (z+profundidade<nz-1):
                    profundidade += 1
                if profundidade > 0:
                    c.append(Camada(profundidade=profundidade,velocidade=velocidade,densidade=rho))
                z += profundidade + 1
            c2D.append(c)
            x += 1
        super().__init__(c2D)

In [10]:
import pickle

def gera_geologia(salva = True, n=10_000, n_colunas=512, nome = 'modelos'):
    print(f'Criando geologias para {nome}')
    m = [ModeloSenoides(nx=n_colunas) for _ in trange(n)]
    if salva:
        with open(f'{nome}.pickle','wb') as file:
            pickle.dump(m, file)
    return m

def carrega_geologia(nome = 'modelos'):
    with open(f'{nome}.pickle','rb') as file:
        m = pickle.load(file)
    return m

In [11]:
def gera_sismica(m, salva = True, nome='sismica'):
    n_linhas = 256
    tempo_aquisicao = 3
    print(f'Criando sísmica completa para {nome}')
    X = [i.adquirir_sismica_2D(t_max=tempo_aquisicao,n_linhas=n_linhas) for i in tqdm(m)]
    print(f'Criando sísmica limpa para {nome}')
    y = [i.adquirir_sismica_2D(t_max=tempo_aquisicao,n_linhas=n_linhas, limpo=True) for i in tqdm(m)]
    if salva:
        X_ = np.expand_dims(np.array(X), -1)
        y_ = np.expand_dims(np.array(y), -1)
        np.savez_compressed(nome, X=X_, y=y_)
    return X, y

In [12]:
def gera_conjunto_completo():
    m_treino = gera_geologia(n=100_000, salva=False, nome='conjunto_treino')
    gera_sismica(m=m_treino,nome='conjunto_treino')
    
    m_teste = gera_geologia(n=30_000, salva=False,nome='conjunto_teste')
    gera_sismica(m=m_teste,nome='conjunto_teste')
    
    m_val = gera_geologia(n=30_000, salva=False,nome='conjunto_val')
    gera_sismica(m=m_val,nome='conjunto_val')
    return

In [ ]:
gera_conjunto_completo()

Criando geologias para conjunto_treino


  0%|          | 0/100000 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Criando sísmica completa para conjunto_treino


In [ ]:
def gera_100k_n(n = 100_000, n_loops=10, salva = True, nome='conjunto_treino'):
    X = None
    y = None
    for i in trange(n_loops):
        print(i)
        m = gera_geologia(n=n//n_loops, nome=f'modelos_{i}', salva=False)
#       gera_sismica(m=m,nome=f'sismica_{i}')
        X_, y_ = gera_sismica(m=m,nome=f'sismica_{i}')
        if X:
            X += X_
            y += y_
        else:
            X = X_
            y = y_
    if salva:
        X = np.expand_dims(np.array(X), -1)
        y = np.expand_dims(np.array(y), -1)
        np.savez_compressed(nome, X=X, y=y)
    return #X, y

In [ ]:
dados_treino = None
for i in range(10):
    arq_treino = f'sismica_{i}.npz'
    dados = np.load(arq_treino)
    if not dados_treino:
        dados_treino = {}
        dados_treino['X'] = dados['X']
        dados_treino['y'] = dados['y']
    else:
        dados_treino['X'] = np.vstack((dados_treino['X'],dados['X']))
        dados_treino['y'] = np.vstack((dados_treino['y'],dados['y']))
dados_teste = None
for i in range(3):
    arq_teste = f'sismica_teste_{i}.npz'
    dados = np.load(arq_teste)
    if not dados_teste:
        dados_teste = {}
        dados_teste['X'] = dados['X']
        dados_teste['y'] = dados['y']
    else:
        dados_teste['X'] = np.vstack((dados_teste['X'],dados['X']))
        dados_teste['y'] = np.vstack((dados_teste['y'],dados['y']))
dados_val = None
for i in range(3):
    arq_val = f'sismica_val_{i}.npz'
    dados = np.load(arq_val)
    if not dados_val:
        dados_val = {}
        dados_val['X'] = dados['X']
        dados_val['y'] = dados['y']
    else:
        dados_val['X'] = np.vstack((dados_val['X'],dados['X']))
        dados_val['y'] = np.vstack((dados_val['y'],dados['y']))